In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("duolingo_reviews_cleaned.csv")

In [ ]:
#vectorization using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['content'])

In [ ]:
#apply LDA for thematic analysis with 4 themes
lda = LatentDirichletAllocation(
n_components=4, 
random_state=42,
learning_method="online"
)
lda.fit(X)

In [ ]:
# get the top words in each theme
terms = vectorizer.get_feature_names_out()
themes = {}
for idx, topic in enumerate(lda.components_):
    top_terms = [terms[i] for i in topic.argsort()[-5:]] # top 5 words
    themes[f"Theme {idx+1}"] = top_terms